# 볼린저 밴드 함수화 
1. 첫번째 함수를 생성 (매개변수 데이터프레임 하나)
    1. 결측치와 이상치를 제거
    2. 컬럼을 종가만 두고 모두 삭제
    3. 이동평균선, 상단 밴드, 하단 밴드 파생변수 생성
2. 두번째 함수를 생성 (매개변수 데이터프레임 하나)
    1. 거래 내역이라는 파생변수 생성 "" <-비어있는 형태로
    2. 볼린저 밴드의 거래 내역을 추가
    3. trade print
3. 세번째 함수를 생성(매개변수 데이터프레임 하나)
    1. 수익률 파생변수 생성 "1"
    2. 판매를 한 날의 수익률 변경
    3. 누적수익률 계산하여 새로운 파생변수를 생성
    4. 최종 누적수익률을 출력(print)

In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from datetime import datetime

import platform
plt.rc('font', family = 'Malgun Gothic')

In [37]:
#Moon_sol 

# 1번 함수 생성

# 수정종가든, 종가든 다 되게 해보자
# 매개변수 추가 col


# 매개변수 추가 start , end


def create_band(df, col, start, end) :

    #인덱스를 시계열로 변경
    df.index = pd.to_datetime(df.index)

    # start, end를 시계열로 변경
    start = datetime.strptime(start, '%Y%m%d').isoformat()
    end = datetime.strptime(end, '%Y%m%d').isoformat()

    # 데이터를 시작시간부터 종료시간까지 필터
    df = df.loc[start : end]



    df = df.loc[~df.isin([np.nan, np.inf, -np.inf]).any(axis='columns'), [col]]
    df['center'] = df.rolling(20).mean()
    df['ub'] = df['center'] + (2* df[col].rolling(20).std())
    df['lb'] = df['center'] - (2* df[col].rolling(20).std())
    
    return df 




In [38]:
_df = pd.read_csv('../csv/GM.csv', index_col='Date')

In [39]:
_df1 = create_band(_df, 'Close', '20110101', '20150101')


In [43]:
_df2 = create_band(_df, 'Adj Close', '20110101', '20150101')


In [29]:
_df.iloc[18:25]

,Close,center,ub,lb
Date,,,,
2010-12-15,33.610001,NaN,NaN,NaN
2010-12-16,33.610001,34.0570,34.942749,33.171251
2010-12-17,34.000000,34.0475,34.931316,33.163684
2010-12-20,33.759998,34.0225,34.909289,33.135711
2010-12-21,33.849998,34.0110,34.900610,33.121390
2010-12-22,34.919998,34.0945,34.996765,33.192235
2010-12-23,34.810001,34.1610,35.068603,33.253397


In [30]:
# 2번째 함수 생성

def add_trade(df) :
    # 기준이 되는 컬럼이 무엇인가 ? 
    # 기준이 되는 컬럼은 컬럼 중에 첫번째 이기 때문 : df.columns[0]
    
    col = df.columns[0]

    # trade 파생변수 생성 
    df['trade'] = ""
    for i in df.index : 
    # 상단 밴드보다 종가가 높은 경우
        if df.loc[i, col] > df.loc[i, 'ub']:
        # 현재 구매 상태이면(전날 'trade'가 'buy')
            if df.shift(1).loc[i, 'trade'] == 'buy' : 
            #매도
                df.loc[i, 'trade'] = ''
            else : 
                df.loc[i, 'trade'] = ''
    # 하단 밴드보다 종가가 낮은 경우
        elif df.loc[i, col] < df.loc[i, 'lb'] : 
        # 현재 구매상태이면,
            if df.shift(1).loc[i, 'trade'] == 'buy' :
            # 구매 상태를 유지 
                df.loc[i, 'trade'] = 'buy'
            else : 
            #매수
                df.loc[i, 'trade'] = 'buy'

        else :
        # 현재 구매상태이면
            if df.shift(1).loc[i , 'trade'] == 'buy' : 
                df.loc[i, 'trade'] = 'buy'
            else :
                df.loc[i, 'trade'] = ''
    
      
    return df

In [41]:
add_trade(_df1)['trade'].value_counts()

buy    559
       447
Name: trade, dtype: int64

In [45]:
_df1

,Close,center,ub,lb,trade
Date,,,,,
2011-01-03,37.060001,NaN,NaN,NaN,
2011-01-04,37.900002,NaN,NaN,NaN,
2011-01-05,38.070000,NaN,NaN,NaN,
2011-01-06,38.900002,NaN,NaN,NaN,
2011-01-07,38.980000,NaN,NaN,NaN,
...,...,...,...,...,...
2014-12-24,33.430000,32.5620,34.464492,30.659507,buy
2014-12-26,33.730000,32.6450,34.601199,30.688801,buy
2014-12-29,34.599998,32.7035,34.821803,30.585196,buy


In [44]:
add_trade(_df2)['trade'].value_counts()

buy    547
       459
Name: trade, dtype: int64

In [33]:
# 세번째 함수 생성

def add_rtn(df) :

    col = df.columns[0]

    
    rtn = 1.0
    df['return'] = 1
    buy = 0.0
    sell = 0.0

    for i in df.index :
    # 구매가 출력
        if (df.shift(1).loc[i, 'trade'] == '') and (df.loc[i, 'trade'] == 'buy') :
            buy = df.loc[i, col]
        
    # 판매가 출력
        elif (df.shift(1).loc[i, 'trade'] == 'buy') and (df.loc[i, 'trade'] == '') :
            sell = df.loc[i, col]
            rtn = (sell-buy) / buy + 1
            df.loc[i, 'return'] = rtn
           

    # 구매가, 판매가 초기화 
        if df.loc[i, 'trade'] == '':
            buy = 0.0
            sell = 0.0


    acc_rtn = 1.0

    for i in df.index :
        rtn = df.loc[i, 'return']
        acc_rtn *= rtn
        df.loc[i, 'acc_rtn'] = acc_rtn

    

    print('누적수익률 : ', acc_rtn)
    return df

In [35]:
add_rtn(_df2)

누적수익률 :  1.1480914103747033


,Close,center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2010-11-18,34.189999,NaN,NaN,NaN,,1.0,1.000000
2010-11-19,34.259998,NaN,NaN,NaN,,1.0,1.000000
2010-11-22,34.080002,NaN,NaN,NaN,,1.0,1.000000
2010-11-23,33.250000,NaN,NaN,NaN,,1.0,1.000000
2010-11-24,33.480000,NaN,NaN,NaN,,1.0,1.000000
...,...,...,...,...,...,...,...
2019-06-18,36.700001,35.45400,37.253548,33.654451,buy,1.0,1.148091
2019-06-19,36.779999,35.43650,37.173130,33.699870,buy,1.0,1.148091
2019-06-20,36.959999,35.50700,37.372713,33.641286,buy,1.0,1.148091


In [51]:
import bollinger as bg 

import imp
imp.reload(bg)

C:\Users\zino0\AppData\Local\Temp\ipykernel_25164\1307585516.py:3: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


<module 'bollinger' from 'c:\\Users\\zino0\\Documents\\GitHub\\Practice_ubion\\Code_Moon\\230413\\bollinger.py'>

In [52]:
df = pd.read_csv('../csv/MSFT.csv', index_col= 'Date')

In [53]:
# 1번 함수 실행

data = bg.create_band(df, 'Adj Close', '20100101', '20190101')

In [56]:
# 2번 함수 실행

data2 = bg.add_trade(data)

In [57]:
# 3번 함수 실행

bg.add_rtn(data2)

누적수익률 :  2.619028014666416


,Adj Close,center,ub,lb,trade,return,acc_rtn
Date,,,,,,,
2010-01-04,24.525019,24.056307,24.964478,23.148135,,1.0,1.000000
2010-01-05,24.532942,24.095134,25.015633,23.174635,,1.0,1.000000
2010-01-06,24.382378,24.133962,25.032815,23.235109,,1.0,1.000000
2010-01-07,24.128809,24.168828,25.004852,23.332803,,1.0,1.000000
2010-01-08,24.295214,24.206467,24.989876,23.423058,,1.0,1.000000
...,...,...,...,...,...,...,...
2018-12-24,93.384033,105.437251,114.470702,96.403801,buy,1.0,2.619028
2018-12-26,99.763077,105.144093,114.525564,95.762621,buy,1.0,2.619028
2018-12-27,100.378166,104.848455,114.447882,95.249027,buy,1.0,2.619028
